In [2215]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

### ETAPA 1: COSECHA ###

In [2216]:
# parametros cosecha
periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
lotes = []
penalizaciones = {
    'C1': [0.03, 0.02, 0.01, 0.005],
    'C2': [0.025, 0.012, 0.004, 0.002],
    'C3': [0.03, 0.02, 0.01, 0.005],
    'C4': [0.025, 0.012, 0.01, 0.002],
    'C5': [0.03, 0.02, 0.01, 0.005],
    'C6': [0.025, 0.012, 0.004, 0.002]
}

In [2217]:
# Leo la base de datos y la guardo en una lista
c.execute("SELECT * FROM lotes")
lotes_obtenidos = c.fetchall()
lotes.append(lotes_obtenidos)
lotes = lotes[0]

In [2218]:
#Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

# pasar lotes a lista de listas
lotes = [list(elem) for elem in lotes]
# Funcion que estima el dia de cosecha con media 0 y desv est 2
def dia_estimado():
        optimal_day = np.random.normal(0, 2)
        return int(round(optimal_day))
for lote in lotes:
    variacion_optimo = dia_estimado()
    lote[5]+=variacion_optimo
    lote.append(variacion_optimo)


In [2219]:
def calcular_probabilidad_lluvia(P_seco_a_lluvioso, P_lluvioso_a_lluvioso):
    # Inicializar probabilidades iniciales según el estado inicial deseado
    P_dia_seco = 1
    P_dia_lluvioso = 0
    dias = 7

    # Calcular probabilidades de estado para cada día
    for _ in range(dias):
        P_dia_seco_nuevo = (P_dia_seco * (1 - P_seco_a_lluvioso) + P_dia_lluvioso * (1 - P_lluvioso_a_lluvioso))
        P_dia_lluvioso_nuevo = (P_dia_seco * P_seco_a_lluvioso + P_dia_lluvioso * ( P_lluvioso_a_lluvioso))
        P_dia_seco, P_dia_lluvioso = P_dia_seco_nuevo, P_dia_lluvioso_nuevo

    porcentaje_lluvia = P_dia_lluvioso 
    
    return porcentaje_lluvia

In [2220]:
for lote in lotes:
    prob_lluvia = calcular_probabilidad_lluvia(lote[6],lote[7])
    prob_lluvia = round(prob_lluvia, 6)
    lote.append(prob_lluvia)

In [2221]:
# Lotes actualizados
print(lotes)

[[1, 'L_1_C4', 1, 'C4', 217.5, 121, 0.43, 0.39, 0.257, 3, 0.413462], [2, 'L_2_C4', 2, 'C4', 517.5, 128, 0.35, 0.44, 0.269, 5, 0.384615], [3, 'L_3_C6', 3, 'C6', 450.0, 77, 0.46, 0.38, 0.362, -2, 0.425926], [4, 'L_4_C4', 4, 'C4', 168.75, 123, 0.32, 0.51, 0.276, 1, 0.395058], [5, 'L_5_C6', 5, 'C6', 243.75, 104, 0.49, 0.6, 0.348, 0, 0.550562], [6, 'L_6_C3', 6, 'C3', 150.0, 93, 0.31, 0.56, 0.579, -3, 0.413308], [7, 'L_7_C5', 7, 'C5', 558.75, 111, 0.43, 0.39, 0.795, 2, 0.413462], [8, 'L_8_C6', 8, 'C6', 442.5, 78, 0.38, 0.35, 0.36, 0, 0.368932], [9, 'L_9_C1', 9, 'C1', 300.0, 124, 0.36, 0.35, 0.528, -1, 0.356436], [10, 'L_10_C5', 10, 'C5', 108.75, 80, 0.5, 0.56, 0.806, 1, 0.531915], [11, 'L_11_C1', 11, 'C1', 555.0, 99, 0.31, 0.53, 0.557, -1, 0.397426], [12, 'L_12_C1', 12, 'C1', 412.5, 125, 0.41, 0.33, 0.516, 1, 0.37963], [13, 'L_13_C5', 13, 'C5', 547.5, 93, 0.5, 0.48, 0.811, 4, 0.490196], [14, 'L_14_C2', 14, 'C2', 498.75, 118, 0.44, 0.52, 0.416, 2, 0.478261], [15, 'L_15_C1', 15, 'C1', 101.25, 

In [2222]:
# Función para predecir el clima con markov
def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
    clima_prediccion = []  # Lista para almacenar la predicción
    tipo_3 = []
    # Inicializar el primer día como seco
    clima_prediccion.append(0)
    tipo_3.append(0)
    # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
    for _ in range(7):
        if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
            clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
            tipo_3.append(0)
        else:  # Si el día anterior fue lluvioso
            clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
            if clima_prediccion[-1] == 1:
                tipo_3.append(1 if random.random() <= 0.05 else 0)
    if sum(tipo_3) >= 1:
        perdida = True
    else:
        perdida = False
    return clima_prediccion, perdida

In [2223]:
def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
    lista_penalizaciones = penalizaciones[tipo_uva]
    index = 7
    while index != 0:
        if index == 7:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[0])
        elif index == 6:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[1])
        elif index == 5:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[2])
        elif index == 4:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 3:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 2:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 1:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        index -= 1
    return calidad

def obtener_penalizaciones_2(tipo_uva, calidad, clima_prediccion):
    lista_penalizaciones = penalizaciones[tipo_uva]
    index = 5
    while index != 0:
        if index == 5:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[0])
        elif index == 4:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[1])
        elif index == 3:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[2])
        elif index == 2:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 1:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        index -= 1
    return calidad



In [2224]:
# funcion que entrega el valor final de la calidad considerando la lluvia
def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
    t_minus = -7
    t_plus = 7
    A = np.array([
        [t_minus**2, t_minus, 1],
        [0, 0, 1],
        [t_plus**2, t_plus, 1]])
    b = np.array([q_t_minus, 1, q_t_plus])
    coefficients = np.linalg.solve(A, b)
    a, b, c = coefficients
    def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
        # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
        calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
        calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
        return calidad_final
    q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
    return q_t_optimal

def calcular_calidad_2(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
    t_minus = -7
    t_plus = 7
    A = np.array([
        [t_minus**2, t_minus, 1],
        [0, 0, 1],
        [t_plus**2, t_plus, 1]])
    b = np.array([q_t_minus, 1, q_t_plus])
    coefficients = np.linalg.solve(A, b)
    a, b, c = coefficients
    def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
        # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
        calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
        calidad_final = obtener_penalizaciones_2(tipo_uva, calidad_sin_lluvia, clima_prediccion)
        return calidad_final
    q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
    return q_t_optimal

In [2225]:
# Parametros para encontrar en la simulacion etapa 1
calidades_lotes = []
lotes_perdidos = []
lotes_obtenidos = []
litros_totales_perdidos = 0
litros_totales_obtenidos = 0
botellas_totales_perdidas = 0
botellas_totales_obtenidas = 0
costos_totales_perdidos = 0
costos_totales_obtenidos = 0

In [2226]:
#Se crea ua funcion que nos entregue una lista que corresponde a la cantidad de lluvia maxima tolerada por cada una de las cepas
#Esta funcion se puede alterar dependiendo de que % de lotes se quiere comprar con compra spot
def tolerancia_cepa(lotes):
    cepas = ["C1","C2","C3","C4","C5","C6"]
    tolerancia = []

    penalizaciones = {'C1': [0.03, 0.02, 0.01, 0.005],
                        'C2': [0.025, 0.012, 0.004, 0.002],
                        'C3': [0.03, 0.02, 0.01, 0.005],
                        'C4': [0.025, 0.012, 0.01, 0.002],
                        'C5': [0.03, 0.02, 0.01, 0.005],
                        'C6': [0.025, 0.012, 0.004, 0.002]
                          }
    #Se hacen dos litas por cada cepa, donde una corresponde a la calidad o ponterador de calidad obtenido en base a una de las partes de la ecuacion de calidad
    #(la parte que pensaliza por la presencia de lluvias en cada uno de los dias previos)
    #Luego de eso se determina un porcentaje de los lotes que será comprado con contrato spot (Ej: el 20 porciento que contenga a los lotes con peor calidad)
    for cepa in cepas:
        valores_lluvia = []
        valores_calidad = []
        for lote in lotes:
            porcentaje_lluvia = lote[10]
            lote_cepa = lote[3]
            if cepa == lote_cepa:
                calidad = ((1-penalizaciones[cepa][0]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][1]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][2]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][3]*porcentaje_lluvia) *
                    (1-penalizaciones[cepa][3]*porcentaje_lluvia)
                    )
                valores_lluvia.append(porcentaje_lluvia)
                valores_calidad.append(calidad)
        
        #Se ordenan las listas de mayor a menor para luego poder obtener que valor corresponde al % deseado
        valores_lluvia.sort(reverse = True)
        valores_calidad.sort(reverse = True)

        #En este caso el 0.2 es porque buscamos la calidad en el 20% mas bajo y con su posicion encontraremos la lluvia que genera la calidad del 20% mas bajo
        posicion_porcentaje = int(len(valores_calidad) * 0.2)
        valor_lluvia = valores_lluvia[posicion_porcentaje]
        tolerancia.append(valor_lluvia)
    return tolerancia             


#Creamos una funcion a la que se le entregue un lote y una lista con el porcentaje de lluvia tolerado por cepa y en base a eso
#se decide que tipo de contrato se escogerá
def tipo_contrato(tolerancia, lote):
    cepa = lote[3]
    porcentaje_lluvia = lote[10]
    if cepa == "C1":
        tolerancia_cepa = tolerancia[0]
        if porcentaje_lluvia >= tolerancia_cepa:
            contrato = 1
        else:
            contrato = 0

    elif cepa == "C2":
        tolerancia_cepa = tolerancia[1]
        if porcentaje_lluvia >= tolerancia_cepa:
            contrato = 1
        else:
            contrato = 0

    elif cepa == "C3":
        tolerancia_cepa = tolerancia[2]
        if porcentaje_lluvia >= tolerancia_cepa:
            contrato = 1
        else:
            contrato = 0
        
    elif cepa == "C4":
        tolerancia_cepa = tolerancia[3]
        if porcentaje_lluvia >= tolerancia_cepa:
            contrato = 1
        else:
            contrato = 0

    elif cepa == "C5":
        tolerancia_cepa = tolerancia[4]
        if porcentaje_lluvia >= tolerancia_cepa:
            contrato = 1
        else:
            contrato = 0

    elif cepa == "C6":
        tolerancia_cepa = tolerancia[5]
        if porcentaje_lluvia >= tolerancia_cepa:
            contrato = 1
        else:
            contrato = 0

    return contrato

In [2227]:
climas_spot = []
climas_spot_3 = []

In [2228]:
# Simulación de la etapa 1

tolerancia_lluvias = tolerancia_cepa(lotes)

for t in range(50, 150):
    for lote in lotes:
        #El siguiente valor corresponde a el contrato que se utilizara en cada lote
        contrato_lote = tipo_contrato(tolerancia_lluvias,lote)
        lote_id = lote[0]
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_dia_optimo = lote[5]
        lote_prob_seco_lluvia = lote[6]
        lote_prob_lluvia_lluvia = lote[7]
        lote_precio_usd = lote[8]
        lote_variacion = lote[9]
        lote_prob_lluvia = lote[10]
        if t == lote_dia_optimo - 8:
            lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
            lista = lote
            q_t = calidades[lote_tipo_uva]
            lote_q_t_minus = q_t[0]
            lote_q_t_plus = q_t[1]
            calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
            calidades_lotes.append(calidad_lote)
            lista.append(calidad_lote)
            lista.append(contrato_lote)
            lista.append(perdida)
            lotes_obtenidos.append(lista)
            if contrato_lote > 0:
                calidad_lote = calcular_calidad_2(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion[:-2], lote_tipo_uva)
                if not perdida:
                    climas_spot.append(calidad_lote)
                elif perdida:
                    climas_spot_3.append(calidad_lote)
                
                
                

In [2229]:
print(lotes_obtenidos)
#lote_id = lote[0]
#lote_cod = lote[1]
#lote_numero = lote[2]
#lote_tipo_uva = lote[3]
#lote_toneladas = lote[4]
#lote_dia_optimo = lote[5]
#ote_prob_seco_lluvia = lote[6]
#lote_prob_lluvia_lluvia = lote[7]
#lote_precio_usd = lote[8]
#lote_variacion = lote[9]
#lote_prob_lluvia = lote[10]
#calidad lote
#contrato lote
#tipo 3(true or false)

[[115, 'L_115_C3', 115, 'C3', 431.25, 69, 0.44, 0.48, 0.609, -3, 0.458333, 0.9735912551020407, 1, False], [175, 'L_175_C1', 175, 'C1', 517.5, 69, 0.41, 0.57, 0.55, -2, 0.488094, 0.9581219494897959, 1, False], [284, 'L_284_C3', 284, 'C3', 360.0, 69, 0.49, 0.6, 0.599, -3, 0.550562, 0.9784836734693877, 1, False], [22, 'L_22_C1', 22, 'C1', 420.0, 70, 0.39, 0.35, 0.554, -2, 0.375, 0.9629366326530612, 0, False], [125, 'L_125_C4', 125, 'C4', 138.75, 71, 0.33, 0.6, 0.272, -2, 0.452008, 0.9593875836734693, 1, False], [152, 'L_152_C5', 152, 'C5', 367.5, 71, 0.46, 0.51, 0.802, -3, 0.484211, 0.924527673619898, 1, False], [91, 'L_91_C3', 91, 'C3', 457.5, 72, 0.49, 0.41, 0.576, -3, 0.453704, 0.9541194299999999, 1, False], [120, 'L_120_C3', 120, 'C3', 217.5, 72, 0.42, 0.46, 0.605, 1, 0.4375, 0.9458703901147959, 0, False], [136, 'L_136_C3', 136, 'C3', 502.5, 72, 0.47, 0.36, 0.591, -1, 0.423424, 0.941117765, 0, False], [238, 'L_238_C5', 238, 'C5', 198.75, 72, 0.32, 0.42, 0.808, 1, 0.355556, 0.933545608

In [2230]:
# Modificamos los lotes obtenidos y perdidos con la info relevante, es decir, 
# sacamos las probabilidades de lluvia y el día óptimo de cosecha, entre otras cosas.
lista_final = []
for lote in lotes_obtenidos:
    lista = []
    lote_cod = lote[1]
    lote_numero = lote[2]
    lote_tipo_uva = lote[3]
    lote_toneladas = lote[4]
    lote_costo = lote[8]
    lote_calidad = lote[11]
    lote_contrato = lote[12]
    lote_optimo = lote[5]
    lote_fermentacion = False
    lote_tipo_3 = lote[13]
    lote_prob_lluvia = lote[10]
    lista.append(lote_cod)
    lista.append(lote_numero)
    lista.append(lote_tipo_uva)
    lista.append(lote_toneladas)
    lista.append(lote_costo)
    lista.append(lote_calidad)
    lista.append(lote_contrato)
    lista.append(lote_optimo)
    lista.append(lote_fermentacion)
    lista.append(lote_tipo_3)
    lista.append(lote_prob_lluvia)  
    lista_final.append(lista)

print(lista_final)
#lote_cod = lote[1]
#lote_numero = lote[2]
#lote_tipo_uva = lote[3]
#lote_toneladas = lote[4]
#lote_costo = lote[8]
#lote_calidad = lote[11]
#lote_contrato = lote[12]
#lote_optimo = lote[5]
#lote_fermentacion = False
#lote_tipo_3 = lote[13]
#lote_prob_lluvia = lote[10]


[['L_115_C3', 115, 'C3', 431.25, 0.609, 0.9735912551020407, 1, 69, False, False, 0.458333], ['L_175_C1', 175, 'C1', 517.5, 0.55, 0.9581219494897959, 1, 69, False, False, 0.488094], ['L_284_C3', 284, 'C3', 360.0, 0.599, 0.9784836734693877, 1, 69, False, False, 0.550562], ['L_22_C1', 22, 'C1', 420.0, 0.554, 0.9629366326530612, 0, 70, False, False, 0.375], ['L_125_C4', 125, 'C4', 138.75, 0.272, 0.9593875836734693, 1, 71, False, False, 0.452008], ['L_152_C5', 152, 'C5', 367.5, 0.802, 0.924527673619898, 1, 71, False, False, 0.484211], ['L_91_C3', 91, 'C3', 457.5, 0.576, 0.9541194299999999, 1, 72, False, False, 0.453704], ['L_120_C3', 120, 'C3', 217.5, 0.605, 0.9458703901147959, 0, 72, False, False, 0.4375], ['L_136_C3', 136, 'C3', 502.5, 0.591, 0.941117765, 0, 72, False, False, 0.423424], ['L_238_C5', 238, 'C5', 198.75, 0.808, 0.93354560828875, 0, 72, False, False, 0.355556], ['L_275_C4', 275, 'C4', 382.5, 0.276, 0.9969816326530612, 0, 72, False, False, 0.402584], ['L_20_C4', 20, 'C4', 225.

Abajo está la recapitulación de la información de la cosecha

In [2231]:
lotes_obtenidos = lista_final
# Encontrar la info que queramos de esta etapa
print(f'Calidad máxima: {max(calidades_lotes)}')
print(f'Calidad mínima: {min(calidades_lotes)}')
print('')
contador = 0
contador_3 = 0
for lote in lista_final:
    lote_cod = lote[0]
    lote_numero = lote[1]
    lote_tipo_uva = lote[2]
    lote_toneladas = lote[3]
    lote_costo = lote[4]
    lote_calidad = lote[5]
    lote_contrato = lote[6]
    lote_optimo = lote[7]
    lote_fermentacion = lote[8]
    lote_tipo_3 = lote[9]
    lote_prob_lluvia = lote[10]
    if not lote_tipo_3:
        kilos = lote_toneladas * 1000
        litros = kilos * 0.5
        litros_totales_obtenidos += litros
        costo_lote = kilos*lote[4]
        if lote[6] == 0:
            costo_lote = costo_lote * 0.8
            costos_totales_obtenidos += costo_lote
        elif lote[6] == 1:
            calidad_spot = climas_spot[contador]
            costos_totales_obtenidos += costo_lote*calidad_spot
            contador += 1
    elif lote_tipo_3:
        kilos = lote_toneladas * 1000
        litros = kilos * 0.5
        litros_totales_perdidos += litros
        costo_lote = kilos*lote[4]
        if lote[6] == 0:
            costo_lote = costo_lote * 0.8
            costos_totales_perdidos += costo_lote
        elif lote[6] == 1:
            costo_lote = costo_lote
            calidad_spot_3 = climas_spot_3[contador_3]
            costos_totales_perdidos += costo_lote*calidad_spot_3
botellas_totales_obtenidas = litros_totales_obtenidos / 0.75
print(f'Litros totales obtenidos: {round(litros_totales_obtenidos)}')
print(f'Botellas totales obtenidas: {round(botellas_totales_obtenidas)}')
print(f'Costos totales esperadas USD (considerando contratos spot y foward): {round(costos_totales_obtenidos)}')
print('')
botellas_totales_perdidas = litros_totales_perdidos / 0.75
print(f'Litros totales perdidos: {round(litros_totales_perdidos)}')
print(f'Botellas totales perdidas: {round(botellas_totales_perdidas)}')
print(f'Costos totales perdidos USD (compra spot): {round(costos_totales_perdidos)}')
print('')
Costos_totales = costos_totales_obtenidos + costos_totales_perdidos
print(f'Costos totales etapa 1: {round(Costos_totales)}')


Calidad máxima: 0.9971853061224489
Calidad mínima: 0.8721110020408163

Litros totales obtenidos: 44859375
Botellas totales obtenidas: 59812500
Costos totales esperadas USD (considerando contratos spot y foward): 39352457

Litros totales perdidos: 3320625
Botellas totales perdidas: 4427500
Costos totales perdidos USD (compra spot): 2662401

Costos totales etapa 1: 42014858


### ETAPA 2: CALIDAD DE UVA ###

In [2232]:
# Parámetros simulación etapa 2
costo_recuperado = 0

In [2233]:
contador = 0

for lote in lista_final:
    lote_cod = lote[0]
    lote_numero = lote[1]
    lote_tipo_uva = lote[2]
    lote_toneladas = lote[3]
    lote_costo = lote[4]
    lote_calidad = lote[5]
    lote_contrato = lote[6]
    lote_optimo = lote[7]
    lote_fermentacion = lote[8]
    lote_tipo_3 = lote[9]
    lote_prob_lluvia = lote[10]
    if lote_tipo_3:
        lote_calidad = 0


    calidad = lote[5]
    cepa = lote[2]
    precio = lote[4]
    kilos = lote[3]*1000
    umbral = umbrales[lote_tipo_uva]
    #Obtenemos costo por lote dependiendo de como ha sido comprado
    ### 1 spot, 0 foward
    if lote_contrato == 0:
        costo = precio * 0.8 * kilos
    elif lote_contrato == 1:
        costo = precio * kilos
    if 0.5 <= lote_calidad < umbral:
        recuperado = costo * 0.3 
    if lote_calidad < 0.5:
        recuperado = costo * 0.05
    else:
        contador += 1
        recuperado = 0
    costo_recuperado += recuperado
Costos_totales -= costo_recuperado

print(f'Costos recuperados etapa 2: {round(costo_recuperado)} USD')
print(f'Costos totales etapa 2, post recuperación: {round(Costos_totales)}')


Costos recuperados etapa 2: 133715 USD
Costos totales etapa 2, post recuperación: 41881144


### ETAPA 3: FERMENTACIÓN ###

In [2234]:
# calcula la duracion en fermentación
def calcular_duracion_vinificacion():
    duracion_aleatoria = random.uniform(7, 9)
    dia_exacto = round(duracion_aleatoria)
    return dia_exacto

In [2235]:
# Parámetros simulación etapa 3
costo_plantas = {1:[1600, 1840, 1840], 2:[1500, 1725, 1725], 3:[1800, 2070, 2070]}
uso_plantas = [[0, 0, 0],[0, 0, 0],[0, 0, 0]]
litros_perdidos = []
costos_perdidos = []
dias_tanques = []


In [2236]:
def espacio_tanques(uso_plantas):
    espacio_disponible = 0
    for planta in uso_plantas:
        if sum(planta) < 72:
            espacio = 72 - sum(planta)
            espacio_disponible += espacio
    return espacio_disponible

In [2237]:
def entrar_tanques(litros, costo):
    botellas = 0
    litros_sobrantes = 0
    condicion = False
    while not condicion:
        if litros > 25000:
            litros -= 25000
            botellas += 1
        else:
            litros_sobrantes = litros
            litros_perdidos.append(litros_sobrantes)
            costo_perdido = costo * (litros_sobrantes/0.5)
            costos_perdidos.append(costo_perdido)
            condicion = True
    return botellas

In [2238]:
def uso_tanques(botellas):
    while botellas > 0:
        if uso_plantas[1][0] < 24:
            uso_plantas[1][0] += 1
            botellas -= 1
        elif uso_plantas[0][0] < 24:
            uso_plantas[0][0] += 1
            botellas -= 1
        elif uso_plantas[1][1] < 24:
            uso_plantas[1][1] += 1
            botellas -= 1
        elif uso_plantas[1][2] < 24:
            uso_plantas[1][2] += 1
            botellas -= 1
        elif uso_plantas[2][0] < 24:
            uso_plantas[2][0] += 1
            botellas -= 1
        elif uso_plantas[0][1] < 24:
            uso_plantas[0][1] += 1
            botellas -= 1
        elif uso_plantas[0][2] < 24:
            uso_plantas[0][2] += 1
            botellas -= 1
        elif uso_plantas[2][1] < 24:
            uso_plantas[2][1] += 1
            botellas -= 1
        elif uso_plantas[2][2] < 24:
            uso_plantas[2][2] += 1
            botellas -= 1
        else:
            break
    return botellas

In [2239]:
def sacar_tanques(botellas):
    while botellas > 0:
        if uso_plantas[1][0] > 0:
            uso_plantas[1][0] -= 1
            botellas -= 1
        elif uso_plantas[0][0] > 0:
            uso_plantas[0][0] -= 1
            botellas -= 1
        elif uso_plantas[1][1] > 0:
            uso_plantas[1][1] -= 1
            botellas -= 1
        elif uso_plantas[1][2] > 0:
            uso_plantas[1][2] -= 1
            botellas -= 1
        elif uso_plantas[2][0] > 0:
            uso_plantas[2][0] -= 1
            botellas -= 1
        elif uso_plantas[0][1] > 0:
            uso_plantas[0][1] -= 1
            botellas -= 1
        elif uso_plantas[0][2] > 0:
            uso_plantas[0][2] -= 1
            botellas -= 1
        elif uso_plantas[2][1] > 0:
            uso_plantas[2][1] -= 1
            botellas -= 1
        elif uso_plantas[2][2] > 0:
            uso_plantas[2][2] -= 1
            botellas -= 1
        else:
            break

In [2240]:
# Más parámetros
lista_fermentacion = []
lista_espera = []
litros_perdidos = []

In [2241]:
for t in range(30, 150):
    for lote in lista_final:
        lote_cod = lote[0]
        lote_numero = lote[1]
        lote_tipo_uva = lote[2]
        lote_toneladas = lote[3]
        lote_costo = lote[4]
        lote_calidad = lote[5]
        lote_contrato = lote[6]
        lote_dia_cosecha = lote[7]
        lote_salida_fermentacion = lote[8]
        lote_tipo_3 = lote[9]
        lote_duracion = calcular_duracion_vinificacion()
        if t == lote_dia_cosecha:
            if lote_tipo_3:
                # los lotes que tuvieron lluvias tipo 3 se pierden y no entran a fermentación
                pass
            elif not lote_tipo_3:
                ### tomar en consideración que cuando dice botellas me refiero a tanques de 
                ### 25000 litros cada uno.
                litros_uva = lote_toneladas * 1000 * 0.5
                botellas = entrar_tanques(litros_uva, lote_costo)
                botellas_restantes = uso_tanques(botellas)
                botellas_fermentadas = botellas - botellas_restantes
                lista = [t + lote_duracion, botellas]
                dias_tanques.append(lista)
                if botellas_restantes > 0:
                    print(f'Se perdieron {botellas_restantes} tanques el día {t}')
                    print(botellas_restantes)
                    costo_perdido = lote_costo * ((botellas_restantes*25000)/0.5)
                    print(f'Con el costo de {costo_perdido} USD')
                    print('')
                    costos_perdidos.append(costo_perdido)
                lote[8] = lote_duracion + t
                litros_fermentados = botellas_fermentadas * 25000
                lista_fermentacion.append([lote_tipo_uva, litros_fermentados, t + lote_duracion])
        if lote_salida_fermentacion == t:
            for dias in dias_tanques:
                if dias[0] == t:
                    botella = dias[1]
                    sacar_tanques(botella)


In [2242]:
# lista para mandar a barricas
print(lista_fermentacion)

[['C3', 200000, 77], ['C1', 250000, 78], ['C3', 175000, 77], ['C1', 200000, 77], ['C4', 50000, 80], ['C5', 175000, 79], ['C3', 225000, 81], ['C3', 100000, 80], ['C3', 250000, 80], ['C5', 75000, 80], ['C4', 175000, 80], ['C4', 100000, 81], ['C5', 75000, 81], ['C4', 250000, 81], ['C6', 100000, 81], ['C4', 250000, 82], ['C3', 175000, 81], ['C2', 225000, 82], ['C2', 150000, 82], ['C6', 250000, 81], ['C2', 50000, 83], ['C2', 150000, 84], ['C3', 250000, 84], ['C5', 200000, 85], ['C2', 275000, 84], ['C5', 100000, 84], ['C2', 50000, 84], ['C3', 125000, 84], ['C2', 50000, 84], ['C6', 200000, 85], ['C2', 175000, 86], ['C1', 175000, 85], ['C6', 200000, 86], ['C2', 125000, 85], ['C4', 225000, 87], ['C1', 200000, 86], ['C3', 50000, 85], ['C3', 50000, 86], ['C2', 100000, 86], ['C3', 200000, 87], ['C1', 75000, 87], ['C1', 125000, 86], ['C4', 50000, 87], ['C6', 200000, 86], ['C6', 75000, 87], ['C5', 50000, 88], ['C2', 225000, 88], ['C1', 50000, 88], ['C1', 200000, 89], ['C6', 75000, 88], ['C2', 50000,

In [2243]:
costos_perdidos = round(sum(costos_perdidos))
print(f'Costos perdidos por no entrar a tanques: {costos_perdidos}')
Costos_totales += sum([1600, 1840, 1840, 1500, 1725, 1725, 1800, 2070, 2070])
Costos_totales = round(Costos_totales)
print(f'Costos totales hasta etapa 3 fermentación: {Costos_totales}')


Costos perdidos por no entrar a tanques: 3589964
Costos totales hasta etapa 3 fermentación: 41897314


### ETAPA 4: CREACIÓN DE BLENDS Y SATISFACCIÓN DE MERCADO